In [ ]:
from datetime import datetime, timedelta
import pandas as pd
import matplotlib.pyplot as plt
import jenkspy
import numpy as np
import seaborn as sns
import chart_studio.plotly as py
import plotly.offline as pyoff
import plotly.graph_objs as go
plt.rcParams['font.family'] = 'Noto Sans Mono CJK TC'
pyoff.init_notebook_mode()

In [ ]:
tx_data = pd.read_csv("C:\\Users\\GK\\Part4.csv")

In [ ]:
tx_data.head(1)

In [ ]:
tx_data['結帳日期'] = pd.to_datetime(tx_data['結帳日期'])

In [ ]:
tx_data['結帳日期'].describe()

In [ ]:
tx_ciie = tx_data

In [ ]:
tx_user = pd.DataFrame(tx_data['帳戶名稱'].unique())
tx_user.columns = ['帳戶名稱']

# Recency

In [ ]:
tx_max_purchase = tx_ciie.groupby('帳戶名稱').結帳日期.max().reset_index()

In [ ]:
tx_max_purchase.columns = ['帳戶名稱','MaxPurchaseDate']

In [ ]:
tx_max_purchase['Recency'] = (tx_max_purchase['MaxPurchaseDate'].max() - tx_max_purchase['MaxPurchaseDate']).dt.days

In [ ]:
tx_user = pd.merge(tx_user, tx_max_purchase[['帳戶名稱','Recency']], on='帳戶名稱')

In [ ]:
tx_user.head()

In [ ]:
tx_user.Recency.describe()

In [ ]:
plot_data = [
    go.Histogram(
        x=tx_user['Recency']
    )
]

plot_layout = go.Layout(
        title='Recency'
    )
fig = go.Figure(data=plot_data, layout=plot_layout)
pyoff.iplot(fig)

# Jenks natural breaks 
Compute "natural breaks" (Fisher-Jenks algorithm) on list / tuple / array / numpy.ndarray of integers/floats.

In [ ]:
def goodness_of_variance_fit(array, classes):
    # get the break points
    classes = jenkspy.jenks_breaks(array, classes)

    # do the actual classification
    classified = np.array([classify(i, classes) for i in array])

    # max value of zones
    maxz = max(classified)

    # nested list of zone indices
    zone_indices = [[idx for idx, val in enumerate(classified) if zone + 1 == val] for zone in range(maxz)]

    # sum of squared deviations from array mean
    sdam = np.sum((array - array.mean()) ** 2)

    # sorted polygon stats
    array_sort = [np.array([array[index] for index in zone]) for zone in zone_indices]

    # sum of squared deviations of class means
    sdcm = sum([np.sum((classified - classified.mean()) ** 2) for classified in array_sort])

    # goodness of variance fit
    gvf = (sdam - sdcm) / sdam

    return gvf

def classify(value, breaks):
    for i in range(1, len(breaks)):
        if value < breaks[i]:
            return i
    return len(breaks) - 1

In [ ]:
array_org = np.array(tx_user[['Recency']].T)[0]
list_GVF = [] 
gvf = 0.0
nclasses = 2
while gvf < .99:
    gvf = goodness_of_variance_fit(array_org, nclasses)
    print(nclasses,gvf)
    list_GVF = list_GVF + [gvf]
    nclasses += 1
plt.figure()
plt.xlim(2,len(list_GVF)+1)
plt.plot([x+2 for x in range(len(list_GVF))], list_GVF)
plt.plot([x+2 for x in range(len(list_GVF))], [0.95]*len(list_GVF))
plt.xlabel("Number of cluster")
plt.show()

In [ ]:
from sklearn.cluster import KMeans
sse={}
tx_recency = tx_user[['Recency']]
for k in range(1, 10):
    kmeans = KMeans(n_clusters=k, max_iter=1000).fit(tx_recency)
    tx_recency["clusters"] = kmeans.labels_
    sse[k] = kmeans.inertia_ 
plt.figure()
plt.plot(list(sse.keys()), list(sse.values()))
plt.xlabel("Number of cluster")
plt.show()

In [ ]:
kmeans = KMeans(n_clusters=6)
kmeans.fit(tx_user[['Recency']])
tx_user['RecencyCluster'] = kmeans.predict(tx_user[['Recency']])

In [ ]:
tx_user.groupby('RecencyCluster')['Recency'].describe()

In [ ]:
def order_cluster(cluster_field_name, target_field_name,df,ascending):
    new_cluster_field_name = 'new_' + cluster_field_name
    df_new = df.groupby(cluster_field_name)[target_field_name].mean().reset_index()
    df_new = df_new.sort_values(by=target_field_name,ascending=ascending).reset_index(drop=True)
    df_new['index'] = df_new.index
    df_final = pd.merge(df,df_new[[cluster_field_name,'index']], on=cluster_field_name)
    df_final = df_final.drop([cluster_field_name],axis=1)
    df_final = df_final.rename(columns={"index":cluster_field_name})
    return df_final


In [ ]:
tx_user = order_cluster('RecencyCluster', 'Recency',tx_user,False)

# Frequency

In [ ]:
tx_frequency = tx_ciie.groupby('帳戶名稱').結帳日期.count().reset_index()

In [ ]:
tx_frequency.columns = ['帳戶名稱','Frequency']

In [ ]:
tx_frequency.head()

In [ ]:
tx_user = pd.merge(tx_user, tx_frequency, on='帳戶名稱')

In [ ]:
tx_user.head()

In [ ]:
tx_user.Frequency.describe()

In [ ]:
plot_data = [
    go.Histogram(
        x=tx_user.query('Frequency < 1000')['Frequency']
    )
]

plot_layout = go.Layout(
        title='Frequency'
    )
fig = go.Figure(data=plot_data, layout=plot_layout)
pyoff.iplot(fig)

In [ ]:
array_org = np.array(tx_user[['Frequency']].T)[0]
list_GVF = [] 
gvf = 0.0
nclasses = 2
while gvf < .99:
    gvf = goodness_of_variance_fit(array_org, nclasses)
    print(nclasses,gvf)
    list_GVF = list_GVF + [gvf]
    nclasses += 1
plt.figure()
plt.xlim(2,len(list_GVF)+1)
plt.plot([x+2 for x in range(len(list_GVF))], list_GVF)
plt.plot([x+2 for x in range(len(list_GVF))], [0.95]*len(list_GVF))
plt.xlabel("Number of cluster")
plt.show()

In [ ]:
sse={}
tx_frequency = tx_user.iloc[:,3].to_frame()
for k in range(1, 10):
    kmeans = KMeans(n_clusters=k, max_iter=1000).fit(tx_frequency)
    tx_frequency["clusters"] = kmeans.labels_
    sse[k] = kmeans.inertia_ 
plt.figure()
plt.plot(list(sse.keys()), list(sse.values()))
plt.xlabel("Number of cluster")
plt.show()

In [ ]:
kmeans = KMeans(n_clusters=6)
kmeans.fit(tx_user[['Frequency']])
tx_user['FrequencyCluster'] = kmeans.predict(tx_user[['Frequency']])

In [ ]:
tx_user.groupby('FrequencyCluster')['Frequency'].describe()

In [ ]:
tx_user = order_cluster('FrequencyCluster', 'Frequency',tx_user,True)

In [ ]:
tx_ciie.columns

# Monetary Value

In [ ]:
tx_revenue = tx_ciie.groupby('帳戶名稱').總價.sum().reset_index()

In [ ]:
tx_revenue.head()

In [ ]:
tx_user = pd.merge(tx_user, tx_revenue, on='帳戶名稱')

In [ ]:
tx_user.總價.describe()

In [ ]:
plot_data = [
    go.Histogram(
        x=tx_user.query('總價 < 4.005100e+06')['總價']
    )
]

plot_layout = go.Layout(
        title='Monetary Value'
    )
fig = go.Figure(data=plot_data, layout=plot_layout)
pyoff.iplot(fig)

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
array_org = np.array(tx_user[['總價']].T)[0]
list_GVF = [] 
gvf = 0.0
nclasses = 2
while gvf < .84:
    gvf = goodness_of_variance_fit(array_org, nclasses)
    print(nclasses,gvf)
    list_GVF = list_GVF + [gvf]
    nclasses += 1
plt.figure()
plt.xlim(2,len(list_GVF))
plt.plot([x+2 for x in range(len(list_GVF))], list_GVF)
plt.plot([x+2 for x in range(len(list_GVF))], [0.8]*len(list_GVF))
plt.xlabel("Number of cluster")
plt.show()

In [ ]:
sse={}
tx_revenue = tx_user[['總價']]
for k in range(1, 10):
    kmeans = KMeans(n_clusters=k, max_iter=1000).fit(tx_revenue)
    tx_revenue["clusters"] = kmeans.labels_
    sse[k] = kmeans.inertia_ 
plt.figure()
plt.plot(list(sse.keys()), list(sse.values()))
plt.xlabel("Number of cluster")
plt.show()

In [ ]:
kmeans = KMeans(n_clusters=6)
kmeans.fit(tx_user[['總價']])
tx_user['RevenueCluster'] = kmeans.predict(tx_user[['總價']])

In [ ]:
tx_user = order_cluster('RevenueCluster', '總價',tx_user,True)

In [ ]:
tx_user.groupby('RevenueCluster')['總價'].describe()

# Overall Segmentation

In [ ]:
tx_user

In [ ]:
tx_user['OverallScore'] = tx_user['RecencyCluster'] + tx_user['FrequencyCluster'] + tx_user['RevenueCluster']

In [ ]:
tx_user.groupby('OverallScore').nunique()

In [ ]:
tx_user.groupby('OverallScore')['Recency','Frequency','總價'].mean()

In [ ]:
tx_user['Segment'] = 'Low-Value'
tx_user.loc[tx_user['OverallScore']>6,'Segment'] = 'Mid-Value' 
tx_user.loc[tx_user['OverallScore']>12,'Segment'] = 'High-Value' 

In [ ]:
tx_graph = tx_user.query("總價 < 5000000 and Frequency < 2000")

plot_data = [
    go.Scatter(
        x=tx_graph.query("Segment == 'Low-Value'")['Frequency'],
        y=tx_graph.query("Segment == 'Low-Value'")['總價'],
        mode='markers',
        name='Low',
        marker= dict(size= 7,
            line= dict(width=1),
            color= 'blue',
            opacity= 0.8
           )
    ),
        go.Scatter(
        x=tx_graph.query("Segment == 'Mid-Value'")['Frequency'],
        y=tx_graph.query("Segment == 'Mid-Value'")['總價'],
        mode='markers',
        name='Mid',
        marker= dict(size= 9,
            line= dict(width=1),
            color= 'green',
            opacity= 0.5
           )
    ),
        go.Scatter(
        x=tx_graph.query("Segment == 'High-Value'")['Frequency'],
        y=tx_graph.query("Segment == 'High-Value'")['總價'],
        mode='markers',
        name='High',
        marker= dict(size= 11,
            line= dict(width=1),
            color= 'red',
            opacity= 0.9
           )
    ),
]

plot_layout = go.Layout(
        yaxis= {'title': "Frequency"},
        xaxis= {'title': "總價"},
        title='Segments'
    )
fig = go.Figure(data=plot_data, layout=plot_layout)
pyoff.iplot(fig)

In [ ]:
tx_graph = tx_user.query("Recency < 5000000 and Frequency < 2000")

plot_data = [
    go.Scatter(
        x=tx_graph.query("Segment == 'Low-Value'")['Recency'],
        y=tx_graph.query("Segment == 'Low-Value'")['Frequency'],
        mode='markers',
        name='Low',
        marker= dict(size= 7,
            line= dict(width=1),
            color= 'blue',
            opacity= 0.8
           )
    ),
        go.Scatter(
        x=tx_graph.query("Segment == 'Mid-Value'")['Recency'],
        y=tx_graph.query("Segment == 'Mid-Value'")['Frequency'],
        mode='markers',
        name='Mid',
        marker= dict(size= 9,
            line= dict(width=1),
            color= 'green',
            opacity= 0.5
           )
    ),
        go.Scatter(
        x=tx_graph.query("Segment == 'High-Value'")['Recency'],
        y=tx_graph.query("Segment == 'High-Value'")['Frequency'],
        mode='markers',
        name='High',
        marker= dict(size= 11,
            line= dict(width=1),
            color= 'red',
            opacity= 0.9
           )
    ),
]

plot_layout = go.Layout(
        yaxis= {'title': "Recency"},
        xaxis= {'title': "Frequency"},
        title='Segments'
    )
fig = go.Figure(data=plot_data, layout=plot_layout)
pyoff.iplot(fig)

In [ ]:
tx_graph = tx_user.query("總價 < 5000000 and Recency < 2000")

plot_data = [
    go.Scatter(
        x=tx_graph.query("Segment == 'Low-Value'")['總價'],
        y=tx_graph.query("Segment == 'Low-Value'")['Recency'],
        mode='markers',
        name='Low',
        marker= dict(size= 7,
            line= dict(width=1),
            color= 'blue',
            opacity= 0.8
           )
    ),
        go.Scatter(
        x=tx_graph.query("Segment == 'Mid-Value'")['總價'],
        y=tx_graph.query("Segment == 'Mid-Value'")['Recency'],
        mode='markers',
        name='Mid',
        marker= dict(size= 9,
            line= dict(width=1),
            color= 'green',
            opacity= 0.5
           )
    ),
        go.Scatter(
        x=tx_graph.query("Segment == 'High-Value'")['總價'],
        y=tx_graph.query("Segment == 'High-Value'")['Recency'],
        mode='markers',
        name='High',
        marker= dict(size= 11,
            line= dict(width=1),
            color= 'red',
            opacity= 0.9
           )
    ),
]

plot_layout = go.Layout(
        yaxis= {'title': "總價"},
        xaxis= {'title': "Recency"},
        title='Segments'
    )
fig = go.Figure(data=plot_data, layout=plot_layout)
pyoff.iplot(fig)

# Policy
High Value: Improve Retention

Mid Value: Improve Retention + Increase Frequency

Low Value: Increase Frequency

In [ ]:
import plotly.express as px

In [ ]:
tx_s = tx_user[(tx_user['Recency']!=7)|(tx_user['Frequency']!=538)][['Recency','Frequency','總價','Segment','OverallScore']]
tx_s = tx_s[(tx_s['Recency']!=665)|(tx_s['Frequency']!=4)][['Recency','Frequency','總價','Segment','OverallScore']]

tx_s['Recency'] = (tx_s['Recency']-tx_s['Recency'].min())/(tx_s['Recency'].max()-tx_s['Recency'].min())
tx_s['Frequency'] = (tx_s['Frequency']-tx_s['Frequency'].min())/(tx_s['Frequency'].max()-tx_s['Frequency'].min())
tx_s['總價'] =(tx_s['總價']-tx_s['總價'].min())/(tx_s['總價'].max()-tx_s['總價'].min())

In [ ]:
df = tx_s[tx_s['Segment']=="Low-Value"]
fig = px.scatter_3d(df, x='Recency', y='Frequency', z='總價',color='OverallScore'
              ,symbol='Segment', opacity=0.7)

# tight layout
fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))

# 巄來

In [ ]:
tx_Long = tx_user[["Recency","Frequency","總價"]]

In [ ]:
tx_Long

In [ ]:
sse={}
tx_revenue = np.array(tx_Long)
for k in range(1, 10):
    kmeans = KMeans(n_clusters=k, max_iter=1000).fit(tx_revenue)
#     tx_revenue["clusters"] = kmeans.labels_
    sse[k] = kmeans.inertia_ 
plt.figure()
plt.plot(list(sse.keys()), list(sse.values()))
plt.xlabel("Number of cluster")
plt.show()

In [ ]:
kmeans = KMeans(n_clusters=6, random_state=0).fit(np.array(tx_Long))
tx_Long["clusters"] = kmeans.labels_

In [ ]:
tx_s = tx_Long[(tx_Long['Recency']!=7)|(tx_Long['Frequency']!=538)][['Recency','Frequency','總價','clusters']]
tx_s = tx_s[(tx_s['Recency']!=665)|(tx_s['Frequency']!=4)][['Recency','Frequency','總價','clusters']]
tx_s['color'] = tx_s['clusters'].astype(str)

df = tx_s#[tx_s['Segment']=="Low-Value"]
fig = px.scatter_3d(df, x='Recency', y='Frequency', z='總價',color='color'
              ,symbol='clusters', opacity=0.7)

# tight layout
fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))

In [ ]:
tx_Long.to_csv("Long.csv",index_label=False)